In [1]:
! git clone https://github.com/VivianZhao12/Multimodal-Sentiment-Analysis-with-Sarcasm-Detection.git

fatal: destination path 'Multimodal-Sentiment-Analysis-with-Sarcasm-Detection' already exists and is not an empty directory.


In [2]:
import sklearn
import imblearn
import pandas as pd
import numpy as np
import seaborn as sns
#import scikitplot as skplt
import statsmodels.api as sm
import random

from sklearn.svm import SVC  # Importing Support Vector Classifier
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE  # For oversampling
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

import torch
from torch import nn
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, BatchNormalization, Layer, Softmax, Activation, Conv1D, MaxPooling1D, Flatten, LSTM, InputLayer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras.backend as K

In [ ]:
df_sar = pd.read_csv('../datasets/sarcasm_final.csv')
df_sar.columns

Index(['Unnamed: 0', 'sarcasm_label', 'emotion_label', 'sentiment_label', 'id',
       'spectral_centroid', 'spectral_bandwidth', 'pitch', 'PCA_MFCC_1',
       'PCA_MFCC_2', 'PCA_MFCC_3', 'PCA_MFCC_4', 'PCA_MFCC_5', 'PCA_MFCC_6',
       'PCA_MFCC_7', 'PCA_MFCC_8', 'sentence_level_similarity_emotion',
       'sentence_level_similarity_word', 'exclamation', 'PCA_W2V_1',
       'PCA_W2V_2', 'PCA_W2V_3', 'PCA_W2V_4', 'PCA_W2V_5', 'PCA_W2V_6',
       'PCA_W2V_7', 'PCA_W2V_8', 'PCA_W2V_9', 'PCA_W2V_10', 'PCA_W2V_11',
       'PCA_W2V_12', 'PCA_W2V_13', 'PCA_W2V_14', 'PCA_W2V_15', 'PCA_W2V_16',
       'PCA_W2V_17', 'PCA_W2V_18', 'PCA_W2V_19', 'PCA_W2V_20', 'PCA_W2V_21',
       'PCA_W2V_22', 'PCA_W2V_23', 'PCA_W2V_24', 'PCA_W2V_25', 'PCA_W2V_26',
       'PCA_W2V_27', 'PCA_W2V_28', 'PCA_W2V_29', 'PCA_W2V_30', 'PCA_W2V_31',
       'PCA_W2V_32', 'PCA_W2V_33', 'PCA_W2V_34', 'PCA_W2V_35', 'PCA_W2V_36',
       'PCA_W2V_37', 'PCA_W2V_38', 'PCA_W2V_39', 'PCA_W2V_40', 'PCA_W2V_41',
       'PCA_W2V_42', 

In [4]:
# Define all the feature columns (text + audio features)
all_feature_columns = [
    'sentence_level_similarity_emotion', 'sentence_level_similarity_word', 'exclamation'
] + [f'PCA_W2V_{i}' for i in range(1, 9)] + [
    'spectral_centroid', 'spectral_bandwidth', 'pitch'
] + [f'PCA_MFCC_{i}' for i in range(1, 9)]

# Select a subset of text features for the text-only model
feature_columns_text = [
    'sentence_level_similarity_emotion', 'sentence_level_similarity_word', 'exclamation'
] + [f'PCA_W2V_{i}' for i in range(1, 9)]

# Select a subset of audio features for the text-only model
feature_columns_audio = [
    'spectral_centroid', 'spectral_bandwidth', 'pitch'
] + [f'PCA_MFCC_{i}' for i in range(1, 9)]

In [5]:
# target prediction
target_column = 'sarcasm_label'

# Prepare the feature set and target column for both models
X_all = df_sar[all_feature_columns]
y = df_sar[target_column]

# Function to scale, split, and apply SMOTE
def preprocess_data(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    return X_train_resampled, X_test, y_train_resampled, y_test

# --- Text-Only Model ---
X_text = df_sar[feature_columns_text]
X_train_text, X_test_text, y_train_text, y_test_text = preprocess_data(X_text, y)

# --- Audio-Only Model --- this is for following hybrid model
X_audio = df_sar[feature_columns_audio]
X_train_audio, X_test_audio, y_train_audio, y_test_audio = preprocess_data(X_audio, y)

In [6]:
# Logistic Regression with text features
logistic_model_text = LogisticRegression(max_iter=500, random_state=42)
logistic_model_text.fit(X_train_text, y_train_text)

# Predict on the test set
y_pred_text = logistic_model_text.predict(X_test_text)

# --- Combined Model (Text + Audio Features) ---
X_train_combined, X_test_combined, y_train_combined, y_test_combined = preprocess_data(X_all, y)

# Logistic Regression with both text and audio features
logistic_model_combined = LogisticRegression(max_iter=500, random_state=42)
logistic_model_combined.fit(X_train_combined, y_train_combined)

# Predict on the test set
y_pred_combined = logistic_model_combined.predict(X_test_combined)
y_pred_logistic_proba = logistic_model_combined.decision_function(X_test_combined)

In [7]:
# Evaluation Results (Text-Only Model)
accuracy_text = accuracy_score(y_test_text, y_pred_text)
cm_text = confusion_matrix(y_test_text, y_pred_text)
cr_text = classification_report(y_test_text, y_pred_text, output_dict=True)

# Evaluation Results (Combined Model)
accuracy_combined = accuracy_score(y_test_combined, y_pred_combined)
cm_combined = confusion_matrix(y_test_combined, y_pred_combined)
cr_combined = classification_report(y_test_combined, y_pred_combined, output_dict=True)

In [8]:
# Assuming X_text.columns contains the correct feature names
feature_names = ['Intercept'] + list(X_all.columns)

# Add an intercept to the combined dataset
X_train_combined_with_intercept = sm.add_constant(X_train_combined)

# Fit the logistic regression model using statsmodels
logit_model = sm.Logit(y_train_combined, X_train_combined_with_intercept)
result = logit_model.fit()

# Convert the result summary to a DataFrame and replace the index with feature names
summary_table = result.summary2().tables[1]  # Get the coefficient table
summary_table.index = feature_names  # Replace generic 'x1', 'x2', ... with actual names

sorted_summary = summary_table.sort_values(by='P>|z|')

# Identify significant features based on p-value
significant_features = summary_table[summary_table['P>|z|'] < 0.01]

# Sort significant features by the absolute value of their coefficients
significant_features_sorted = significant_features.reindex(
    significant_features['Coef.'].abs().sort_values(ascending=False).index
)


Optimization terminated successfully.
         Current function value: 0.318500
         Iterations 8


In [9]:
# Support Vector Machine (SVM) Model
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train_combined, y_train_combined)

# Predict with SVM
y_pred_svm = svm_model.predict(X_test_combined)
# Calculate probabilities for SVM
y_pred_svm_proba = svm_model.decision_function(X_test_combined)  # For SVM, use decision_function for Precision-Recall


# Evaluate SVM Model
print("\nSupport Vector Machine (SVM) Results:")
print(f"Accuracy: {accuracy_score(y_test_combined, y_pred_svm):.2f}")
print("Classification Report:\n", classification_report(y_test_combined, y_pred_svm))


Support Vector Machine (SVM) Results:
Accuracy: 0.79
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.82      0.85        38
           1       0.59      0.71      0.65        14

    accuracy                           0.79        52
   macro avg       0.74      0.77      0.75        52
weighted avg       0.81      0.79      0.79        52



In [10]:
# Naive Bayes Model
nb_model = GaussianNB()
nb_model.fit(X_train_combined, y_train_combined)
y_pred_nb_proba = nb_model.predict(X_test_combined)

# Evaluate Naive Bayes Model
print("\nNaive Bayes Results:")
print(f"Accuracy: {accuracy_score(y_test_combined, y_pred_nb_proba):.2f}")
print("Classification Report:\n", classification_report(y_test_combined, y_pred_nb_proba))


Naive Bayes Results:
Accuracy: 0.77
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.76      0.83        38
           1       0.55      0.79      0.65        14

    accuracy                           0.77        52
   macro avg       0.73      0.77      0.74        52
weighted avg       0.81      0.77      0.78        52



In [11]:
# Random Forest Model
clf_model = RandomForestClassifier(n_estimators=100, random_state=42)
clf_model.fit(X_train_combined, y_train_combined)

# Making predictions on the test set
y_pred_clf = clf_model.predict(X_test_combined)

# Calculating the accuracy
accuracy_clf = accuracy_score(y_test_combined, y_pred_clf)

# Generating a classification report
class_report = classification_report(y_test_combined, y_pred_clf)

# Printing out the results
print("\nRandom Forest Results:")
print(f"Accuracy: {accuracy_clf:.2f}")
print("Classification Report:\n", class_report)


Random Forest Results:
Accuracy: 0.81
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87        38
           1       0.67      0.57      0.62        14

    accuracy                           0.81        52
   macro avg       0.76      0.73      0.74        52
weighted avg       0.80      0.81      0.80        52



In [12]:
# Calculate accuracies for each model
accuracy_logistic = logistic_model_combined.score(X_test_combined, y_test_combined)
accuracy_svm = svm_model.score(X_test_combined, y_test_combined)
accuracy_nb = nb_model.score(X_test_combined, y_test_combined)

# Print accuracy values
print(f"Logistic Regression Accuracy: {accuracy_logistic:.2f}")
print(f"SVM Accuracy: {accuracy_svm:.2f}")
print(f"Naive Bayes Accuracy: {accuracy_nb:.2f}")
print(f"Random Forest Accuracy: {accuracy_clf:.2f}")

Logistic Regression Accuracy: 0.77
SVM Accuracy: 0.79
Naive Bayes Accuracy: 0.77
Random Forest Accuracy: 0.81


In [13]:
# Logistic Regression Precision-Recall
precision_logistic, recall_logistic, _ = precision_recall_curve(y_test_combined, y_pred_logistic_proba)
pr_auc_logistic = auc(recall_logistic, precision_logistic)

# SVM Precision-Recall
precision_svm, recall_svm, _ = precision_recall_curve(y_test_combined, y_pred_svm_proba)
pr_auc_svm = auc(recall_svm, precision_svm)

# Naive Bayes Precision-Recall
precision_nb, recall_nb, _ = precision_recall_curve(y_test_combined, y_pred_nb_proba)
pr_auc_nb = auc(recall_nb, precision_nb)

# Assuming clf_rf is your trained Random Forest classifier
y_pred_rf_proba = clf_model.predict_proba(X_test_combined)[:, 1]
precision_rf, recall_rf, _ = precision_recall_curve(y_test_combined, y_pred_rf_proba)
pr_auc_rf = auc(recall_rf, precision_rf)

# Compute ROC curves and AUC scores
fpr_logistic, tpr_logistic, _ = roc_curve(y_test_combined, y_pred_logistic_proba)
fpr_nb, tpr_nb, _ = roc_curve(y_test_combined, y_pred_nb_proba)
fpr_svm, tpr_svm, _ = roc_curve(y_test_combined, y_pred_svm_proba)

auc_logistic = roc_auc_score(y_test_combined, y_pred_logistic_proba)
auc_svm = roc_auc_score(y_test_combined, y_pred_svm_proba)
auc_nb = roc_auc_score(y_test_combined, y_pred_nb_proba)

fpr_rf, tpr_rf, _ = roc_curve(y_test_combined, y_pred_rf_proba)
auc_rf = roc_auc_score(y_test_combined, y_pred_rf_proba)

In [14]:
# Set a random seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# Ensuring the same random seed is used every time before any layer weights are initialized
# Build a full connect neural network
model_FCNN = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_combined.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_FCNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model_FCNN.fit(X_train_combined, y_train_combined, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
fcnn_loss, fcnn_accuracy = model_FCNN.evaluate(X_test_combined, y_test_combined)
print(f'FCNN Accuracy: {fcnn_accuracy}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 136ms/step - accuracy: 0.5851 - loss: 0.6845 - val_accuracy: 0.0167 - val_loss: 0.8102
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.5604 - loss: 0.6930 - val_accuracy: 0.0500 - val_loss: 0.8005
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6461 - loss: 0.5887 - val_accuracy: 0.1333 - val_loss: 0.7831
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6502 - loss: 0.6126 - val_accuracy: 0.4167 - val_loss: 0.7517
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5983 - loss: 0.6396 - val_accuracy: 0.5833 - val_loss: 0.7159
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.6498 - loss: 0.5753 - val_accuracy: 0.7667 - val_loss: 0.6757
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.7206 - loss: 0.5669 - val_accuracy: 0.8000 - val_loss: 0.6513
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7191 - loss: 0.5607 - val_accuracy: 0.8167 - val_loss: 0.6200

In [15]:
model_FCNN.save("model_FCNN.keras")

In [16]:
# Set a random seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# Build a more robust fully connected neural network
model_ehFCNN = Sequential([
    InputLayer(input_shape=(X_train_combined.shape[1],)),

    Dense(64, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),

    Dense(64, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),

    Dense(1, activation='sigmoid')
])

# Optimizer with a learning rate schedule
optimizer = Adam(learning_rate=0.001)
model_ehFCNN.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and saving the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Train the model with validation split and callbacks
history = model_ehFCNN.fit(
    X_train_combined,
    y_train_combined,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate the model on the test set
fcnn_loss, fcnn_accuracy = model_ehFCNN.evaluate(X_test_combined, y_test_combined)
print(f'Enhanced FCNN Model Accuracy: {fcnn_accuracy}')

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.5478 - loss: 1.6533 - val_accuracy: 0.0500 - val_loss: 1.6955
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5885 - loss: 1.5467 - val_accuracy: 0.3500 - val_loss: 1.6234
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7010 - loss: 1.4232 - val_accuracy: 0.6667 - val_loss: 1.5498
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7476 - loss: 1.4069 - val_accuracy: 0.8000 - val_loss: 1.4839
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7208 - loss: 1.3477 - val_accuracy: 0.8667 - val_loss: 1.4236
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7443 - loss: 1.2948 - val_accuracy: 0.9000 - val_loss: 1.3587
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7943 - loss: 1.2591 - val_accuracy: 0.9167 - val_loss: 1.3038
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8329 - loss: 1.1972 - val_accuracy: 0.9333 - val_loss: 1.2512
Epoch 9/50


In [17]:
# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

def build_hybrid_model(num_text_features, num_audio_features, regularization_rate=0.01):
    # Text input branch
    text_input = Input(shape=(num_text_features,), name='text_input')
    text_hidden = Dense(128, activation='relu', kernel_regularizer=l2(regularization_rate))(text_input)
    text_hidden = BatchNormalization()(text_hidden)
    text_hidden = Dropout(0.3)(text_hidden)

    # Audio input branch
    audio_input = Input(shape=(num_audio_features,), name='audio_input')
    audio_hidden = Dense(128, activation='relu', kernel_regularizer=l2(regularization_rate))(audio_input)
    audio_hidden = BatchNormalization()(audio_hidden)
    audio_hidden = Dropout(0.3)(audio_hidden)

    # Concatenate both branches
    concatenated = Concatenate()([text_hidden, audio_hidden])
    concatenated = Dense(64, activation='relu', kernel_regularizer=l2(regularization_rate))(concatenated)
    concatenated = Dropout(0.3)(concatenated)

    # Output layer
    output = Dense(1, activation='sigmoid')(concatenated)

    # Build and compile the model
    model = Model(inputs=[text_input, audio_input], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Instantiate the model
model_ehhb = build_hybrid_model(num_text_features=11, num_audio_features=11)

# Callbacks for early stopping and model checkpointing
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

# Assume X_train_text, X_train_audio, y_train_combined are prepared
# and similarly for X_test_text, X_test_audio, y_test_combined
history = model_ehhb.fit(
    [X_train_text, X_train_audio],
    y_train_combined,
    validation_data=([X_test_text, X_test_audio], y_test_combined),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, checkpoint]
)

# Evaluate the model
ehhb_accuracy = model_ehhb.evaluate([X_test_text, X_test_audio], y_test_combined)[1]
print("Hybrid Model Accuracy:", ehhb_accuracy)

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.6228 - loss: 2.2666 - val_accuracy: 0.7308 - val_loss: 2.0007
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8281 - loss: 1.7813 - val_accuracy: 0.7500 - val_loss: 1.8690
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8478 - loss: 1.6895 - val_accuracy: 0.7692 - val_loss: 1.7872
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8988 - loss: 1.5809 - val_accuracy: 0.7885 - val_loss: 1.7248
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8794 - loss: 1.4910 - val_accuracy: 0.7692 - val_loss: 1.6694
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8331 - loss: 1.5130 - val_accuracy: 0.7308 - val_loss: 1.6226
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8981 - loss: 1.3866 - val_accuracy: 0.7500 - val_loss: 1.5783
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9277 - loss: 1.3058 - val_accuracy: 0.7308 - v

# Sentiment Model

In [18]:
df_sen = pd.read_csv('/content/Multimodal-Sentiment-Analysis-with-Sarcasm-Detection/datasets/sentiment_final.csv')
df_sen.columns

Index(['Unnamed: 0', 'sarcasm_label', 'emotion_label', 'sentiment_label', 'id',
       'sentence_level_similarity_emotion', 'sentence_level_similarity_word',
       'exclamation', 'PCA_MFCC_1', 'PCA_MFCC_2', 'PCA_MFCC_3', 'PCA_MFCC_4',
       'PCA_MFCC_5', 'PCA_MFCC_6', 'PCA_MFCC_7', 'PCA_MFCC_8', 'PCA_W2V_1',
       'PCA_W2V_2', 'PCA_W2V_3', 'PCA_W2V_4', 'PCA_W2V_5', 'PCA_W2V_6',
       'PCA_W2V_7', 'PCA_W2V_8', 'PCA_W2V_9', 'PCA_W2V_10', 'PCA_W2V_11',
       'PCA_W2V_12', 'PCA_W2V_13', 'PCA_W2V_14', 'PCA_W2V_15', 'PCA_W2V_16',
       'PCA_W2V_17', 'PCA_W2V_18', 'PCA_W2V_19', 'PCA_W2V_20', 'PCA_W2V_21',
       'PCA_W2V_22', 'PCA_W2V_23', 'PCA_W2V_24', 'PCA_W2V_25', 'PCA_W2V_26',
       'PCA_W2V_27', 'PCA_W2V_28', 'PCA_W2V_29', 'PCA_W2V_30', 'PCA_W2V_31',
       'PCA_W2V_32', 'PCA_W2V_33', 'PCA_W2V_34', 'PCA_W2V_35', 'PCA_W2V_36',
       'PCA_W2V_37', 'PCA_W2V_38', 'PCA_W2V_39', 'PCA_W2V_40', 'PCA_W2V_41',
       'PCA_W2V_42', 'PCA_W2V_43', 'PCA_W2V_44', 'PCA_W2V_45', 'PCA_W2V_46',
   

In [19]:
# Define all the feature columns (text + audio features)
all_feature_columns = [
    'sentence_level_similarity_emotion', 'sentence_level_similarity_word', 'exclamation'
] + [f'PCA_W2V_{i}' for i in range(1, 9)] + [
    'spectral_centroid', 'spectral_bandwidth', 'pitch'
] + [f'PCA_MFCC_{i}' for i in range(1, 9)]

# Select a subset of text features for the text-only model
feature_columns_text = [
    'sentence_level_similarity_emotion', 'sentence_level_similarity_word', 'exclamation'
] + [f'PCA_W2V_{i}' for i in range(1, 9)]

# Select a subset of audio features for the text-only model
feature_columns_audio = [
    'spectral_centroid', 'spectral_bandwidth', 'pitch'
] + [f'PCA_MFCC_{i}' for i in range(1, 9)]

In [20]:
# target prediction
target_column = 'sentiment_label'

# Prepare the feature set and target column for both models
X_all = df_sen[all_feature_columns]
y = df_sen[target_column]

# Function to scale, split, and apply SMOTE
def preprocess_data(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    return X_train_resampled, X_test, y_train_resampled, y_test

In [21]:
# --- Combined Model (Text + Audio Features) ---
X_train_combined, X_test_combined, y_train_combined, y_test_combined = preprocess_data(X_all, y)

In [22]:
from tensorflow.keras.models import load_model
model_FCNN_new = load_model("model_FCNN.keras")

## XG Boost without integrating Sarcasm Model

In [23]:
import xgboost as xgb

# Create XGBoost DMatrix
dtrain = xgb.DMatrix(X_train_combined, label=y_train_combined)
dtest = xgb.DMatrix(X_test_combined, label=y_test_combined)

# Set parameters
params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "max_depth": 5,
    "n_estimators": 10,
    "eta": 0.1,
    "seed": 42
}

# Train the model
model_xgb = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions
preds = model_xgb.predict(dtest)
predictions = [1 if p > 0.5 else 0 for p in preds]

# Evaluate
accuracy = accuracy_score(y_test_combined, predictions)
print(f"XGBoost Model Accuracy: {accuracy}")

XGBoost Model Accuracy: 0.5428571428571428


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:30:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


In [24]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_combined, y_train_combined)

# Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_combined)

accuracy = accuracy_score(y_test_combined, y_pred)
print(f"Optimized XGBoost Accuracy: {accuracy}")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:30:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:30:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:30:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:30:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:30:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

Optimized XGBoost Accuracy: 0.5714285714285714


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:30:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:30:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:30:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


## XG Boost with Sarcasm Model Integrated

In [25]:
# Assume `sarcasm_model` is a trained sarcasm detection model
sarcasm_predictions_train = clf_model.predict(X_train_combined)  # Generate sarcasm labels (0 or 1)
sarcasm_predictions_test = clf_model.predict(X_test_combined)  # Generate sarcasm labels (0 or 1)
# Add sarcasm predictions as a new feature
features_with_sarcasm_train = np.hstack((X_train_combined, sarcasm_predictions_train.reshape(-1, 1)))
features_with_sarcasm_test = np.hstack((X_test_combined, sarcasm_predictions_test.reshape(-1, 1)))

dtrain = xgb.DMatrix(features_with_sarcasm_train, label=y_train_combined)
dtest = xgb.DMatrix(features_with_sarcasm_test, label=y_test_combined)

model_xgb_sarcasm = xgb.train(params, dtrain, num_boost_round=100)
preds = model_xgb_sarcasm.predict(dtest)
predictions = [1 if p > 0.5 else 0 for p in preds]

accuracy = accuracy_score(y_test_combined, predictions)
print(f"Updated XGBoost Model with Sarcasm Accuracy: {accuracy}")

Updated XGBoost Model with Sarcasm Accuracy: 0.5714285714285714


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:30:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


## FFNN without Sarcasm

In [26]:
model_fnn = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=(X_train_combined.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_fnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model_fnn.fit(X_train_combined, y_train_combined, epochs=100, batch_size=5, validation_split=0.2)

# Evaluate the model
loss, accuracy = model_fnn.evaluate(X_test_combined, y_test_combined)
print(f"FNN Model Accuracy: {accuracy}")

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.4331 - loss: 0.8909 - val_accuracy: 0.7188 - val_loss: 0.6463
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5283 - loss: 0.7803 - val_accuracy: 0.6250 - val_loss: 0.6629
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4800 - loss: 0.8008 - val_accuracy: 0.5938 - val_loss: 0.6822
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4814 - loss: 0.7425 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6555 - loss: 0.6299 - val_accuracy: 0.4375 - val_loss: 0.6928
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6791 - loss: 0.6430 - val_accuracy: 0.4375 - val_loss: 0.6929
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5827 - loss: 0.7136 - val_accuracy: 0.4375 - val_loss: 0.7052
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5362 - loss: 0.6989 - val_accuracy: 0.4062 - val_loss: 0.

## FFNN with Sarcasm

In [27]:
# Assume `sarcasm_model` is a trained sarcasm detection model
sarcasm_predictions_train = clf_model.predict(X_train_combined)  # Generate sarcasm labels (0 or 1)
sarcasm_predictions_test = clf_model.predict(X_test_combined)  # Generate sarcasm labels (0 or 1)
# Add sarcasm predictions as a new feature
features_with_sarcasm_train = np.hstack((X_train_combined, sarcasm_predictions_train.reshape(-1, 1)))
features_with_sarcasm_test = np.hstack((X_test_combined, sarcasm_predictions_test.reshape(-1, 1)))

model_fnn = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=(features_with_sarcasm_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_fnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model_fnn.fit(features_with_sarcasm_train, y_train_combined, epochs=100, batch_size=5, validation_split=0.2)

# Evaluate the model
loss, accuracy = model_fnn.evaluate(features_with_sarcasm_test, y_test_combined)
print(f"FNN Model Accuracy: {accuracy}")

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.4370 - loss: 1.0319 - val_accuracy: 0.5938 - val_loss: 0.6867
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5217 - loss: 0.8853 - val_accuracy: 0.4375 - val_loss: 0.7129
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5705 - loss: 0.7275 - val_accuracy: 0.4062 - val_loss: 0.7257
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4869 - loss: 0.9043 - val_accuracy: 0.3438 - val_loss: 0.7262
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5553 - loss: 0.8006 - val_accuracy: 0.3750 - val_loss: 0.7214
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6179 - loss: 0.6462 - val_accuracy: 0.5312 - val_loss: 0.7093
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6446 - loss: 0.7464 - val_accuracy: 0.4062 - val_loss: 0.7081
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6603 - loss: 0.6175 - val_accuracy: 0.4062 - val_loss: 0.

## SVM without Sarcasm

In [28]:
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train_combined, y_train_combined)

# Predict with SVM
y_pred_svm = svm_model.predict(X_test_combined)
# Calculate probabilities for SVM
y_pred_svm_proba = svm_model.decision_function(X_test_combined)  # For SVM, use decision_function for Precision-Recall


# Evaluate SVM Model
print("\nSupport Vector Machine (SVM) Results:")
print(f"Accuracy: {accuracy_score(y_test_combined, y_pred_svm):.2f}")
print("Classification Report:\n", classification_report(y_test_combined, y_pred_svm))


Support Vector Machine (SVM) Results:
Accuracy: 0.51
Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.55      0.56        20
           1       0.44      0.47      0.45        15

    accuracy                           0.51        35
   macro avg       0.51      0.51      0.51        35
weighted avg       0.52      0.51      0.52        35



## Create Sentiment Data

In [ ]:
df_sen = pd.read_csv('../datasets/sentiment_final.csv')
df_sar = pd.read_csv('../datasets/sarcasm_final.csv')
df_comb = pd.concat([df_sar, df_sen], ignore_index=True)

df_comb.columns

Index(['Unnamed: 0', 'sarcasm_label', 'emotion_label', 'sentiment_label', 'id',
       'spectral_centroid', 'spectral_bandwidth', 'pitch', 'PCA_MFCC_1',
       'PCA_MFCC_2', 'PCA_MFCC_3', 'PCA_MFCC_4', 'PCA_MFCC_5', 'PCA_MFCC_6',
       'PCA_MFCC_7', 'PCA_MFCC_8', 'sentence_level_similarity_emotion',
       'sentence_level_similarity_word', 'exclamation', 'PCA_W2V_1',
       'PCA_W2V_2', 'PCA_W2V_3', 'PCA_W2V_4', 'PCA_W2V_5', 'PCA_W2V_6',
       'PCA_W2V_7', 'PCA_W2V_8', 'PCA_W2V_9', 'PCA_W2V_10', 'PCA_W2V_11',
       'PCA_W2V_12', 'PCA_W2V_13', 'PCA_W2V_14', 'PCA_W2V_15', 'PCA_W2V_16',
       'PCA_W2V_17', 'PCA_W2V_18', 'PCA_W2V_19', 'PCA_W2V_20', 'PCA_W2V_21',
       'PCA_W2V_22', 'PCA_W2V_23', 'PCA_W2V_24', 'PCA_W2V_25', 'PCA_W2V_26',
       'PCA_W2V_27', 'PCA_W2V_28', 'PCA_W2V_29', 'PCA_W2V_30', 'PCA_W2V_31',
       'PCA_W2V_32', 'PCA_W2V_33', 'PCA_W2V_34', 'PCA_W2V_35', 'PCA_W2V_36',
       'PCA_W2V_37', 'PCA_W2V_38', 'PCA_W2V_39', 'PCA_W2V_40', 'PCA_W2V_41',
       'PCA_W2V_42', 

In [30]:
# Define all the feature columns (text + audio features)
all_feature_columns = [
    'sentence_level_similarity_emotion', 'sentence_level_similarity_word', 'exclamation'
] + [f'PCA_W2V_{i}' for i in range(1, 9)] + [
    'spectral_centroid', 'spectral_bandwidth', 'pitch'
] + [f'PCA_MFCC_{i}' for i in range(1, 9)]

# Select a subset of text features for the text-only model
feature_columns_text = [
    'sentence_level_similarity_emotion', 'sentence_level_similarity_word', 'exclamation'
] + [f'PCA_W2V_{i}' for i in range(1, 9)]

# Select a subset of audio features for the text-only model
feature_columns_audio = [
    'spectral_centroid', 'spectral_bandwidth', 'pitch'
] + [f'PCA_MFCC_{i}' for i in range(1, 9)]

In [31]:
# target prediction
target_column = 'sarcasm_label'

# Prepare the feature set and target column for both models
X_all = df_comb[all_feature_columns]
y = df_comb[target_column]

# Function to scale, split, and apply SMOTE
def preprocess_data(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    return X_train_resampled, X_test, y_train_resampled, y_test

# --- Text-Only Model ---
X_text = df_comb[feature_columns_text]
X_train_text, X_test_text, y_train_text, y_test_text = preprocess_data(X_text, y)

# --- Audio-Only Model --- this is for following hybrid model
X_audio = df_comb[feature_columns_audio]
X_train_audio, X_test_audio, y_train_audio, y_test_audio = preprocess_data(X_audio, y)

In [32]:


# --- Combined Model (Text + Audio Features) ---
X_train_combined, X_test_combined, y_train_combined, y_test_combined = preprocess_data(X_all, y)



## Sentiment Model with Deep Neural Network


In [33]:
import tensorflow as tf

def create_model(max_length, audio_feature_length, binary=True):
    text_input = tf.keras.layers.Input(shape=(max_length,), name='text_input')
    text_output = tf.keras.layers.Dense(128, activation='relu')(text_input)

    audio_input = tf.keras.layers.Input(shape=(audio_feature_length,), name='audio_input')
    audio_output = tf.keras.layers.Dense(128, activation='relu')(audio_input)

    combined = tf.keras.layers.concatenate([text_output, audio_output])
    x = tf.keras.layers.Dense(64, activation='relu')(combined)
    x = tf.keras.layers.Dropout(0.2)(x)

    if binary:
        output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
        loss = 'binary_crossentropy'
    else:
        output = tf.keras.layers.Dense(2, activation='softmax')(x)
        loss = 'categorical_crossentropy'

    model = tf.keras.Model(inputs=[text_input, audio_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    return model

# Create the model for binary classification
model = create_model(max_length=X_train_text.shape[1], audio_feature_length=X_train_audio.shape[1], binary=True)

# Fit the model
history = model.fit(
    [X_train_text, X_train_audio], y_train_text,
    validation_data=([X_test_text, X_test_audio], y_test_text),
    epochs=10,
    batch_size=32
)

# Evaluate the model
results = model.evaluate([X_test_text, X_test_audio], y_test_text)
print(f"Test Accuracy: {results[1]*100:.2f}%")

predicted_probs = model.predict([X_test_text, X_test_audio])
predicted_labels = (predicted_probs > 0.5).astype(int).flatten()

print(f"Accuracy: {accuracy_score(y_test_combined, predicted_labels):.2f}")
print("Classification Report:\n", classification_report(y_test_combined, predicted_labels))


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5828 - loss: 0.6689 - val_accuracy: 0.7674 - val_loss: 0.5526
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7932 - loss: 0.5065 - val_accuracy: 0.7674 - val_loss: 0.4734
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8247 - loss: 0.4381 - val_accuracy: 0.8256 - val_loss: 0.4395
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8294 - loss: 0.3918 - val_accuracy: 0.8488 - val_loss: 0.4255
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8560 - loss: 0.3557 - val_accuracy: 0.8488 - val_loss: 0.4100
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8671 - loss: 0.3198 - val_accuracy: 0.8372 - val_loss: 0.4020
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8918 - loss: 0.2950 - val_accuracy: 0.8372 - val_loss: 0.4042
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8890 - loss: 0.2771 - val_accuracy: 0.8488 - val_loss

## Sentiment Model with Deep Neural Network + Sarcasm as feature


In [34]:
# Assume `sarcasm_model` is a trained sarcasm detection model
sarcasm_predictions_train = clf_model.predict(X_train_combined)  # Generate sarcasm labels (0 or 1)
sarcasm_predictions_test = clf_model.predict(X_test_combined)  # Generate sarcasm labels (0 or 1)
# Add sarcasm predictions as a new feature
features_with_sarcasm_train = np.hstack((X_train_combined, sarcasm_predictions_train.reshape(-1, 1)))
features_with_sarcasm_test = np.hstack((X_test_combined, sarcasm_predictions_test.reshape(-1, 1)))

import tensorflow as tf

def create_model(feature_length, binary=True):
    # Input layer to accommodate text, audio, and sarcasm features
    inputs = tf.keras.layers.Input(shape=(feature_length,), name='combined_input')
    x = tf.keras.layers.Dense(128, activation='relu')(inputs)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    # Output layer configuration based on binary or categorical
    if binary:
        output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
        loss = 'binary_crossentropy'
    else:
        output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
        loss = 'categorical_crossentropy'

    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    return model

# Determine the feature length based on your new combined datasets
feature_length = features_with_sarcasm_train.shape[1]

# Create the model
model = create_model(feature_length, binary=True)

# Fit the model using the enhanced datasets
history = model.fit(
    features_with_sarcasm_train, y_train_text,
    epochs=10,
    batch_size=32
)
# Evaluate the model on the test set
results = model.evaluate(features_with_sarcasm_test, y_test_text)
print(f"Test Accuracy: {results[1]*100:.2f}%")

predicted_probs = model.predict(features_with_sarcasm_test)
predicted_labels = (predicted_probs > 0.5).astype(int).flatten()

print(f"Accuracy: {accuracy_score(y_test_combined, predicted_labels):.2f}")
print("Classification Report:\n", classification_report(y_test_combined, predicted_labels))




Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5525 - loss: 0.7120
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7682 - loss: 0.5112 
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8213 - loss: 0.4267
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8586 - loss: 0.3735 
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8756 - loss: 0.3214 
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8903 - loss: 0.3025 
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9147 - loss: 0.2663 
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9296 - loss: 0.2214
Epoch 9/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9441 - loss: 0.2073
Epoch 10/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9318 - loss: 0.2043 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8950 - loss: 0.2927  
Test Accuracy: 88.37%
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Accuracy: 0.88
Class

## Sentiment Model with Deep Neural Network + Sarcasm Sentiment Set to 0

In [35]:
final_predicted_labels = np.where(sarcasm_predictions_test == 1, 0, predicted_labels)

# Calculate accuracy and other metrics after adjusting for sarcasm
print(f"Adjusted Accuracy: {accuracy_score(y_test_text, final_predicted_labels):.2f}")
print("Adjusted Classification Report:\n", classification_report(y_test_text, final_predicted_labels))
print("Adjusted Confusion Matrix:\n", confusion_matrix(y_test_text, final_predicted_labels))

Adjusted Accuracy: 0.73
Adjusted Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.95      0.85        66
           1       0.00      0.00      0.00        20

    accuracy                           0.73        86
   macro avg       0.38      0.48      0.42        86
weighted avg       0.58      0.73      0.65        86

Adjusted Confusion Matrix:
 [[63  3]
 [20  0]]
